In [20]:
from glob import glob
import pandas as pd
import rasterio


In [36]:
chips_df = pd.read_csv('/workspace/gfm-gap-filling/pretraining/chip_tracker.csv')
tifs = glob('/workspace/chips_binary_filtered/*_merged.tif')
cdls = glob('/workspace/chips_binary_filtered/*mask.tif')

output_folder = ('/workspace/chips_single_band')

In [37]:
print(tifs[1:5])

['/workspace/chips_binary_filtered/chip_034_026_merged.tif', '/workspace/chips_binary_filtered/chip_004_002_merged.tif', '/workspace/chips_binary_filtered/chip_027_010_merged.tif', '/workspace/chips_binary_filtered/chip_004_077_merged.tif']


In [50]:
## chatgpt function
def export_bands(input_path, output_dir, band_names):
    with rasterio.open(input_path) as src:
        #print(src.count)
        for band_idx in range(src.count):
            # Read the band data
            band = src.read(band_idx + 1)
            
            # Define the output file path
            output_path = f"{output_dir}/{band_names[band_idx]}.tif"
            out_meta = src.meta.copy()
            out_meta.update({"count": 1})
            # Create the output file and write the band data
            with rasterio.open(output_path, 'w', **out_meta) as dst:
                dst.write(band, 1)

# Example usage
# input_path = 'path/to/input.tif'  # Path to the input multiband GeoTIFF
# output_dir = 'path/to/output'     # Directory to save the output TIFF files
# band_names = ['red', 'green', 'blue']  # User-defined names for each band

# export_bands(input_path, output_dir, band_names)


In [39]:
def row_to_dict(data_frame, chip_id):
    # Extract the specified row as a pandas Series
    row = data_frame.loc[data_frame.chip_id == chip_id]
    
    # Convert the row to a dictionary
    row_dict = row.to_dict(orient = 'records')[0]
    
    return row_dict



In [43]:



for tif_path in tifs:
    chip_id = tif_path.split('/')[-1][:12]
    chip_meta = row_to_dict(chips_df, chip_id[5:])
    date_1 = str(int(chip_meta['first_image_date']))
    date_2 = str(int(chip_meta['second_image_date']))
    date_3 = str(int(chip_meta['third_image_date']))
    band_names = [f'HLS.S30.{chip_id}.{date_1}.v1.4.B02',
              f'HLS.S30.{chip_id}.{date_1}.v1.4.B03',
              f'HLS.S30.{chip_id}.{date_1}.v1.4.B04',
              f'HLS.S30.{chip_id}.{date_1}.v1.4.B08',
              f'HLS.S30.{chip_id}.{date_2}.v1.4.B02',
              f'HLS.S30.{chip_id}.{date_2}.v1.4.B03',
              f'HLS.S30.{chip_id}.{date_2}.v1.4.B04',
              f'HLS.S30.{chip_id}.{date_2}.v1.4.B08',
              f'HLS.S30.{chip_id}.{date_3}.v1.4.B02',
              f'HLS.S30.{chip_id}.{date_3}.v1.4.B03',
              f'HLS.S30.{chip_id}.{date_3}.v1.4.B04',
              f'HLS.S30.{chip_id}.{date_3}.v1.4.B08']
    export_bands(tif_path, output_folder, band_names)
    
    

In [51]:
## export cdl binary




for tif_path in cdls:
  #  print(tif_path)
    chip_id = tif_path.split('/')[-1][:12]
    chip_meta = row_to_dict(chips_df, chip_id[5:])
  #  print(chip_meta)
    date_1 = str(int(chip_meta['first_image_date']))
    date_2 = str(int(chip_meta['second_image_date']))
    date_3 = str(int(chip_meta['third_image_date']))
    band_names = [f'HLS.S30.{chip_id}.{date_1}.v1.4.CDL']
    try:
        export_bands(tif_path, output_folder, band_names)
    except Exception:
        pass

    band_names = [f'HLS.S30.{chip_id}.{date_2}.v1.4.CDL']
    try:
        export_bands(tif_path, output_folder, band_names)
    except Exception:
        pass

    band_names = [f'HLS.S30.{chip_id}.{date_3}.v1.4.CDL']
    try:
        export_bands(tif_path, output_folder, band_names)
    except Exception:
        pass
    
    

